## In-Context Learning (ICL) with contextual information

In this section, we used the In-Context Learning (ICL) model using additional contextual information. By incorporating general knowledge about geopolymer concrete, we aim to enhance the model's predictive capabilities and achieve better performance than the traditional models. The fine-tuning prompt considers the following factors for geopolymer concrete with FA/GGBFS:

1. Higher GGBFS content (e.g., 0.3/0.7) typically yields greater strength.
2. Lower W/C ratios generally result in increased compressive strength.
3. Enhanced powder content (FA + GGBFS) contributes to higher strength.
4. Curing methods: Heat curing is often more effective for higher FA content (e.g., 0.5/0.5), while GGBFS-rich blends can achieve adequate strength with ambient curing.



The In-Content Learning process involves providing the model with concrete formulations as prompts and their respective compressive strengths as completions for learning. Subsequently, the model will receive only prompts and will be required to generate completions itself. 

The prompt format is as follows:

Please consider the following disclaimer: For geopolymer concrete with FA/GGBFS, consider the following: (1) Higher GGBFS content (e.g., 0.3/0.7) typically yields greater strength. (2) Lower W/C ratios generally result in increased compressive strength. (3) Enhanced powder content (FA + GGBFS) contributes to higher strength. (4) Curing methods: Heat curing is often more effective for higher FA content (e.g., 0.5/0.5), while GGBFS-rich blends can achieve adequate strength with ambient curing.; We will do an exercise where I will provide you with concrete formulations as prompts and their respective respective compressive strength as completions for you to learn from. Then you will only receive prompts and need to complete it yourself. Add the respective Idx to each answer. Let's go:

{training_text}


All training and test sets used for the experiments will be stored in the `results` folder, allowing for easy access and reproducibility of the study.

In [1]:
# Predict Alkali activated concrete properties with in-context learning using openAI's text-davinci-003 model

from openai import OpenAI
from openai import RateLimitError
import random
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import re
import os
import time
from utils import *

# Load OpenAI client
client = OpenAI()
model_name = "gpt-3.5-turbo-instruct"

# Read data from file
data_path = os.path.join('data', 'transformed_data.txt')
with open(data_path, 'r') as f:
    data = f.readlines()    

context_prompt = "Please consider the following disclaimer: For geopolymer concrete with FA/GGBFS, consider the following: (1) Higher GGBFS content (e.g., 0.3/0.7) typically yields greater strength. (2) Lower W/C ratios generally result in increased compressive strength. (3) Enhanced powder content (FA + GGBFS) contributes to higher strength. (4) Curing methods: Heat curing is often more effective for higher FA content (e.g., 0.5/0.5), while GGBFS-rich blends can achieve adequate strength with ambient curing.; We will do an exercise where I will provide you with concrete formulations as prompts and their respective respective compressive strength as completions for you to learn from. Then you will only receive prompts and need to complete it yourself with a single number. Lets go:"
# Randomly sample n lines for training and N lines for testing
n = 10
N = 25

# Initialize empty lists to store results
result_list = []

indices = list(range(len(data)))
# Repeat the process 10 times
for i in range(10):
    #np.random.seed(i)
    random.seed(i)
    test_prompts, true_values, predictions = gather_LLM_results(data,
                                                                n,
                                                                N,
                                                                client,
                                                                model_name,
                                                                indices,
                                                                context_prompt=context_prompt)

    append_to_result_list(test_prompts, true_values, predictions, result_list)


save_results_to_csv(model_name, "ICL_finetuned", result_list)

response: 55.12
result: 55.12
response: 45.88
result: 45.88
response: 41.67
result: 41.67
response: 47.97
result: 47.97
response: 60.71
result: 60.71
response: 60.71
result: 60.71
response: 57.14
result: 57.14
response: 49.12
result: 49.12
response: 50.12
result: 50.12
response: 41.67
result: 41.67
response: 52.08
result: 52.08
response: 41.67
result: 41.67
response: 68.57
result: 68.57
response: 47.62
result: 47.62
response: 41.67
result: 41.67
response: 47.56
result: 47.56
response: 54.12
result: 54.12
response: 57.14
result: 57.14
response: 49.12
result: 49.12
response: 41.67
result: 41.67
response: 60.71
result: 60.71
response: 45.88
result: 45.88
response: 54.12
result: 54.12
response: 41.86
result: 41.86
response: 49.17
result: 49.17
R-squared: 0.13
MAE: 7.96
MSE: 119.52
response: 57.14
result: 57.14
response: 24.91
result: 24.91
response: 33.33
result: 33.33
response: 49.81
result: 49.81
response: 35.91
result: 35.91
response: 24.91
result: 24.91
response: 24.91
result: 24.91
re

### Vanilla In-Context Learning (ICL)

In this section, we will benchmark the performance of the In-Context Learning (ICL) model using the text-davinci-003 model from OpenAI. ICL leverages large language models to incorporate context and general knowledge, providing flexibility in handling non-numeric inputs and overcoming the limitations of traditional vector space formulations. The experiments will be conducted using the OpenAI API without any additional fine-tuning or contextual information.

All training and test sets used for the experiments will be stored in the `results` folder, allowing for easy access and reproducibility of the study.


In [2]:
# Predict Alkali activated concrete properties with in-context learning using openAI's text-davinci-003 model

from openai import OpenAI
import random
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import re
import os
from utils import *

# Load API key
client = OpenAI()
model_name = "gpt-3.5-turbo-instruct"

data_path = os.path.join('data', 'transformed_data.txt')

context_prompt = "We will do an exercise where I will provide you with concrete formulations as prompts and their respective respective compressive strength as completions for you to learn from. Then you will only receive prompts and need to complete it yourself with a single number. Lets go:"

# Read data from file
with open(data_path, 'r') as f:
    data = f.readlines()
# Randomly sample n lines for training and N lines for testing
n = 10
N = 25

# Initialize empty lists to store results
result_list = []
indices = list(range(len(data)))
# Repeat the process 10 times
for i in range(10):
    #np.random.seed(i)
    random.seed(i)
    test_prompts, true_values, predictions = gather_LLM_results(data,
                                                                n,
                                                                N,
                                                                client,
                                                                model_name,
                                                                indices,
                                                                context_prompt=context_prompt)

    append_to_result_list(test_prompts, true_values, predictions, result_list)


save_results_to_csv(model_name, "ICL", result_list)


response: 54.12
result: 54.12
response: 44.44
result: 44.44
response: 41.12
result: 41.12
response: 47.62
result: 47.62
response: 59.12
result: 59.12
response: 59.09
result: 59.09
response: 55.26
result: 55.26
response: 47.12
result: 47.12
response: 41.92
result: 41.92
response: 47.62
result: 47.62
response: 52.94
result: 52.94
response: 41.67
result: 41.67
response: 68.57
result: 68.57
response: 49.41
result: 49.41
response: 47.62
result: 47.62
response: 49.12
result: 49.12
response: 54.12
result: 54.12
response: 54.55
result: 54.55
response: 59.09
result: 59.09
response: 51.11
result: 51.11
response: 41.67
result: 41.67
response: 45.88
result: 45.88
response: 54.12
result: 54.12
response: 41.12
result: 41.12
response: 61.11
result: 61.11
R-squared: -0.57
MAE: 10.64
MSE: 215.33
response: 57.14
result: 57.14
response: 35.29
result: 35.29
response: 41.67
result: 41.67
response: 49.82
result: 49.82
response: 35.12
result: 35.12
response: 35.12
result: 35.12
response: 35.12
result: 35.12


### Gaussian Process Regression (GPR)

In this section, we will benchmark the performance of the Gaussian Process Regression (GPR) model using the `scikit-learn` library. GPR is a non-parametric, Bayesian approach to regression that provides uncertainty estimates of the predictions. It is based on the assumption that any finite set of data points can be modeled by a multivariate Gaussian distribution.

All training and test sets used for the experiments will be stored in the `results` folder, allowing for easy access and reproducibility of the study.



In [3]:
import random
import os
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import re
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import ConstantKernel, Matern
from sklearn.model_selection import ShuffleSplit


#data = pd.read_csv(r'data/numeric_data.csv')
file_path = os.path.join('data', 'numeric_data.csv')
data = pd.read_csv(file_path)


kernel = ConstantKernel(1.0, (1e-3, 1e3)) * Matern(length_scale=10, nu=1.5)

gpr = GaussianProcessRegressor(kernel=kernel)

# DataFrames to store results
train_results_df = pd.DataFrame()
test_results_df = pd.DataFrame(columns=['Iteration', 'Idx_Sample', 'Input Features', 'True Values', 'Predicted Values'])

n = 10
N = 25

indices = list(range(len(data)))

for i in range(10):
    #np.random.seed(i)
    random.seed(i)
    #random_indices = np.random.choice(len(data), n+N, False)
    random.shuffle(indices)
    # Sample the data based on the provided indices
    train_data = data.iloc[indices[:n]]
    test_data = data.iloc[indices[n:n+N]]

    target_column = 'fc_28dGroundTruth'
    idx_column = 'Idx_Sample'
    X_train = train_data.drop(columns=[target_column, idx_column], axis=1)

    # Normalize input features
    X_scaler = StandardScaler()
    X_train = X_scaler.fit_transform(X_train)

    # Scale the target variable for training
    y_scaler = MinMaxScaler()
    y_train = y_scaler.fit_transform(train_data[target_column].copy().to_numpy().reshape(-1, 1))

    gpr.fit(X_train, y_train)

    # Test data
    X_test = test_data.drop(columns=[target_column, idx_column], axis=1)
    X_test = X_scaler.transform(X_test)

    # Predict on test data
    predictions = gpr.predict(X_test)
    predictions = predictions.reshape(-1, 1)
    predictions = y_scaler.inverse_transform(predictions)

    # Store true and predicted values
    true_values = test_data[target_column].copy().to_numpy().reshape(-1, 1)
    idx_sample = test_data[idx_column].copy().to_numpy()

    # Store train data
    #train_results_df = train_results_df.append(train_data)
    train_results_df = pd.concat([train_results_df, train_data], ignore_index=True)

    # Store test data
    iteration_df = pd.DataFrame({
        'Iteration': i+1,
        'Idx_Sample': idx_sample,
        'Input Features': list(X_test),
        'True Values': true_values.flatten(),
        'Predicted Values': predictions.flatten()
    })

    test_results_df = pd.concat([test_results_df, iteration_df], ignore_index=True)

    # Calculate R2 score and mean absolute error
    r2 = r2_score(true_values, predictions)
    mae = mean_absolute_error(true_values, predictions)   
    mse = mean_squared_error(true_values, predictions)

    # Evaluation
    print(f"Iteration: {i+1}")
    print(f"R-squared: {r2:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")



train_results_file = os.path.join('results', model_name, 'GPR', 'train.csv')

# Make needed directories
dir_name = os.path.dirname(train_results_file)
os.makedirs(dir_name, exist_ok=True)

train_results_df.to_csv(train_results_file, index=False)

test_results_file = os.path.join('results', model_name, 'GPR', 'test.csv')
test_results_df.to_csv(test_results_file, index=False)



Iteration: 1
R-squared: 0.78
MAE: 4.35
MSE: 29.76
Iteration: 2
R-squared: 0.01
MAE: 8.30
MSE: 100.57
Iteration: 3
R-squared: 0.62
MAE: 4.34
MSE: 28.64
Iteration: 4
R-squared: 0.35
MAE: 6.41
MSE: 59.51
Iteration: 5
R-squared: 0.28
MAE: 7.41
MSE: 80.64
Iteration: 6
R-squared: 0.64
MAE: 4.89
MSE: 31.87
Iteration: 7
R-squared: 0.78
MAE: 4.45
MSE: 30.08
Iteration: 8
R-squared: 0.69
MAE: 4.66
MSE: 30.84
Iteration: 9
R-squared: 0.48
MAE: 6.17
MSE: 59.95
Iteration: 10
R-squared: 0.65
MAE: 5.70
MSE: 45.49


### Random Forest (M5-Tree with Linear Tree Models)

In this section, we will benchmark the performance of the Random Forest (RF) model using an M5-Tree with linear tree models and well-calibrated uncertainty estimates, implemented in the `lolopy` library. RF is an ensemble learning method that constructs multiple decision trees and combines their output for improved prediction accuracy and reduced overfitting. The M5-Tree with linear tree models enhances the standard RF by incorporating linear regression models in the tree leaves, providing better performance on certain types of data. 

EDIT: No longer uses 'lolopy' library

All training and test sets used for the experiments will be stored in the `results` folder, allowing for easy access and reproducibility of the study.

In [4]:
import random
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import re
import matplotlib.pyplot as plt


file_path = os.path.join('data', 'numeric_data.csv')
data = pd.read_csv(file_path)  
        

# DataFrames to store results
train_results_df = pd.DataFrame()
test_results_df = pd.DataFrame(columns=['Iteration', 'Idx_Sample', 'Input Features', 'True Values', 'Predicted Values'])

n = 10
N = 25

indices = list(range(len(data)))

for i in range(10):
    #np.random.seed(i)
    #random_indices = np.random.choice(range(0, len(data)), n+N, False)

    random.seed(i)
    random.shuffle(indices)
    
    # Sample the data based on the provided indices
    train_data = data.iloc[indices[:n]]
    test_data = data.iloc[indices[n:n+N]]

    target_column = 'fc_28dGroundTruth'
    idx_column = 'Idx_Sample'
    X_train = train_data.drop(columns=[target_column, idx_column], axis=1)

    # Normalize input features
    X_scaler = StandardScaler()
    X_train = X_scaler.fit_transform(X_train)

    # Scale the target variable for training
    y_scaler = MinMaxScaler()
    y_train = y_scaler.fit_transform(train_data[target_column].copy().to_numpy().reshape(-1, 1))

    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)

    # Test data
    X_test = test_data.drop(columns=[target_column, idx_column], axis=1)
    X_test = X_scaler.transform(X_test)

    # Predict on test data
    predictions = rf.predict(X_test)
    predictions = predictions.reshape(-1, 1)
    predictions = y_scaler.inverse_transform(predictions)

    # Store true and predicted values
    true_values = test_data[target_column].copy().to_numpy().reshape(-1, 1)
    idx_sample = test_data[idx_column].copy().to_numpy()

    # Store train data
    #train_results_df = train_results_df.append(train_data)
    train_results_df = pd.concat([train_results_df, train_data], ignore_index=True)

    # Store test data
    iteration_df = pd.DataFrame({
        'Iteration': i+1,
        'Idx_Sample': idx_sample,
        'Input Features': list(X_test),
        'True Values': true_values.flatten(),
        'Predicted Values': predictions.flatten()
    })

    test_results_df = pd.concat([test_results_df, iteration_df], ignore_index=True)

    # Calculate R2 score and mean absolute error
    r2 = r2_score(true_values, predictions)
    mae = mean_absolute_error(true_values, predictions)   
    mse = mean_squared_error(true_values, predictions)

    # Evaluation
    print(f"Iteration: {i+1}")
    print(f"R-squared: {r2:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")


train_results_file = os.path.join('results', model_name, 'RF', 'train.csv')

dir_name = os.path.dirname(train_results_file)
os.makedirs(dir_name, exist_ok=True)

train_results_df.to_csv(train_results_file, index=False)

test_results_file = os.path.join('results', model_name, 'RF', 'test.csv')
test_results_df.to_csv(test_results_file, index=False)

C:\Users\andre\AppData\Local\Temp\ipykernel_6464\2298538285.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
C:\Users\andre\AppData\Local\Temp\ipykernel_6464\2298538285.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
C:\Users\andre\AppData\Local\Temp\ipykernel_6464\2298538285.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


Iteration: 1
R-squared: 0.67
MAE: 5.53
MSE: 45.62
Iteration: 2
R-squared: 0.70
MAE: 4.57
MSE: 30.68
Iteration: 3
R-squared: 0.44
MAE: 4.91
MSE: 41.95


C:\Users\andre\AppData\Local\Temp\ipykernel_6464\2298538285.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
C:\Users\andre\AppData\Local\Temp\ipykernel_6464\2298538285.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
C:\Users\andre\AppData\Local\Temp\ipykernel_6464\2298538285.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


Iteration: 4
R-squared: 0.43
MAE: 6.06
MSE: 52.67
Iteration: 5
R-squared: 0.65
MAE: 4.88
MSE: 39.37
Iteration: 6
R-squared: 0.64
MAE: 4.45
MSE: 32.06


C:\Users\andre\AppData\Local\Temp\ipykernel_6464\2298538285.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
C:\Users\andre\AppData\Local\Temp\ipykernel_6464\2298538285.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
C:\Users\andre\AppData\Local\Temp\ipykernel_6464\2298538285.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


Iteration: 7
R-squared: 0.71
MAE: 5.10
MSE: 39.98
Iteration: 8
R-squared: 0.59
MAE: 5.05
MSE: 40.36
Iteration: 9
R-squared: 0.48
MAE: 6.15
MSE: 59.76
Iteration: 10
R-squared: 0.51
MAE: 6.42
MSE: 64.02


C:\Users\andre\AppData\Local\Temp\ipykernel_6464\2298538285.py:48: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)
